In [1]:
%pip install python-dotenv alpaca-py requests pandas


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%reload_ext dotenv
%dotenv

In [3]:
from alpaca.data.historical.crypto import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
import pandas as pd
from os import getenv, makedirs
from datetime import datetime
import pytz
from itertools import product

In [4]:
# Get supported pairs
import requests
r = requests.get(url='https://api.alpaca.markets/v2/assets?asset_class=crypto', headers={'Apca-Api-Key-Id': getenv('KEY_ID'), 'Apca-Api-Secret-Key': getenv('SECRET_KEY')})
assets = []
for x in r.json():
    assets.append(x["symbol"])
print(len(assets))

45


In [ ]:
client = CryptoHistoricalDataClient(api_key=getenv('KEY_ID'), secret_key=getenv('SECRET_KEY'))
makedirs("./data/",exist_ok=True)
base = datetime.today()
date_list = [pd.Timestamp(base) - pd.DateOffset(months=x) for x in range(0, 12*5)]
for end in date_list:
    start = end - pd.DateOffset(months=1)
    print(start, end)
    req = CryptoBarsRequest(symbol_or_symbols=assets, timeframe=TimeFrame.Minute, start=start, end=end)
    bars = client.get_crypto_bars(request_params=req, feed="us")
    df = bars.df
    df.to_csv(f'./data/{str(start.date()).replace("-","_")}.csv')

2023-04-19 17:50:55.391818 2023-05-19 17:50:55.391818
2023-03-19 17:50:55.391818 2023-04-19 17:50:55.391818
2023-02-19 17:50:55.391818 2023-03-19 17:50:55.391818
2023-01-19 17:50:55.391818 2023-02-19 17:50:55.391818


In [ ]:
# currencies = df.index.get_level_values(0).unique()
# times = df.index.get_level_values(1).unique()
# idx = df.index
# pairs = list(product(currencies, times))
# missing = [x for x in pairs if x not in idx]
# missing_currencies = pd.Series([x[0] for x in missing]).unique().tolist()
# print(missing_currencies)
# df.drop(missing_currencies, level=0, inplace=True)

In [ ]:
# df.to_csv('data.csv')